<a href="https://colab.research.google.com/github/HarshaVardhanBabu/TutorialMaterial/blob/master/Imputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler
from sklearn.datasets import load_boston
import numpy as np
# https://pypi.org/project/fancyimpute/

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
Using TensorFlow backend.


In [0]:
boston = load_boston()
print(boston.data.shape)
X = boston.data
Y = boston.target

X = X.copy()

In [0]:
X.ravel()[np.random.choice(X.size, 100, replace=False)] = np.nan

In [0]:
X_filled_knn = KNN(k=3).fit_transform(X)

Imputing row 1/506 with 1 missing, elapsed time: 0.050
Imputing row 101/506 with 0 missing, elapsed time: 0.051
Imputing row 201/506 with 0 missing, elapsed time: 0.052
Imputing row 301/506 with 1 missing, elapsed time: 0.053
Imputing row 401/506 with 0 missing, elapsed time: 0.053
Imputing row 501/506 with 0 missing, elapsed time: 0.054


In [0]:
# matrix completion using convex optimization to find low-rank solution
# that still matches observed values. Slow!
X_filled_nnm = NuclearNormMinimization().fit_transform(X)

----------------------------------------------------------------------------
	SCS v2.1.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 180786
eps = 1.00e-04, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 148096, constraints m = 161252
Cones:	primal zero / dual free vars: 6578
	linear vars: 19734
	sd vars: 134940, sd blks: 1
Setup time: 2.35e-01s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 5.20e+21  4.89e+21  1.00e+00 -7.99e+26  1.45e+26  1.96e+26  1.24e-01 
   100| 7.62e-04  8.19e-04  2.78e-04  1.75e+04  1.75e+04  1.82e-12  2.94e+01 
   200| 6.85e-04  6.88e-04  1.57e-04  1.75e+04  1

In [0]:
# Instead of solving the nuclear norm objective directly, instead
# induce sparsity using singular value thresholding
X_incomplete_normalized = BiScaler().fit_transform(X)
X_filled_softimpute = SoftImpute().fit_transform(X_incomplete_normalized)

[BiScaler] Initial log residual value = 14.884686
[BiScaler] Iter 1: log residual = 3.166649, log improvement ratio=11.718036
[BiScaler] Iter 2: log residual = 2.470413, log improvement ratio=0.696237
[BiScaler] Iter 3: log residual = 1.792303, log improvement ratio=0.678110
[BiScaler] Iter 4: log residual = 1.150422, log improvement ratio=0.641881
[BiScaler] Iter 5: log residual = 0.510987, log improvement ratio=0.639435
[BiScaler] Iter 6: log residual = -0.133191, log improvement ratio=0.644178
[BiScaler] Iter 7: log residual = -0.786211, log improvement ratio=0.653020
[BiScaler] Iter 8: log residual = -1.448650, log improvement ratio=0.662439
[BiScaler] Iter 9: log residual = -2.119371, log improvement ratio=0.670721
[BiScaler] Iter 10: log residual = -2.796614, log improvement ratio=0.677243
[BiScaler] Iter 11: log residual = -3.478584, log improvement ratio=0.681971
[BiScaler] Iter 12: log residual = -4.163702, log improvement ratio=0.685117
[BiScaler] Iter 13: log residual = -4.8

In [0]:
missing_mask = ~np.isnan(X)

(array([[ True,  True,  True, ...,  True,  True, False],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]]),
 array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
                nan],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]]))

In [0]:
# print mean squared error for the  imputation methods above
nnm_mse = ((X_filled_nnm[missing_mask] - X[missing_mask]) ** 2).mean()
print("Nuclear norm minimization MSE: %f" % nnm_mse)

softImpute_mse = ((X_filled_softimpute[missing_mask] - X[missing_mask]) ** 2).mean()
print("SoftImpute MSE: %f" % softImpute_mse)

knn_mse = ((X_filled_knn[missing_mask] - X[missing_mask]) ** 2).mean()
print("knnImpute MSE: %f" % knn_mse)

Nuclear norm minimization MSE: 0.000000
SoftImpute MSE: 25663.819201
knnImpute MSE: 0.000000
